## PyPSA power flow

### Imports and Warnings Configuration

In [7]:
import pypsa
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Network Initialization and Bus Creation

In [8]:
network = pypsa.Network()

n_buses = 3

for i in range(n_buses):
    network.add("Bus", "My bus {}".format(i), v_nom=20.)

print(network.buses)

attribute  v_nom type    x    y carrier unit  v_mag_pu_set  v_mag_pu_min  \
Bus                                                                        
My bus 0    20.0       0.0  0.0      AC                1.0           0.0   
My bus 1    20.0       0.0  0.0      AC                1.0           0.0   
My bus 2    20.0       0.0  0.0      AC                1.0           0.0   

attribute  v_mag_pu_max control generator sub_network  
Bus                                                    
My bus 0            inf      PQ                        
My bus 1            inf      PQ                        
My bus 2            inf      PQ                        


### Line Creation

In [9]:
for i in range(n_buses):
    network.add("Line", "My line {}".format(i),
               bus0="My bus {}".format(i),
               bus1="My bus {}".format((i+1) % n_buses),
               x=0.1, 
               r=0.01)

print(network.lines)

attribute      bus0      bus1 type    x     r    g    b  s_nom  s_nom_mod  \
Line                                                                        
My line 0  My bus 0  My bus 1       0.1  0.01  0.0  0.0    0.0        0.0   
My line 1  My bus 1  My bus 2       0.1  0.01  0.0  0.0    0.0        0.0   
My line 2  My bus 2  My bus 0       0.1  0.01  0.0  0.0    0.0        0.0   

attribute  s_nom_extendable  ...  v_ang_min  v_ang_max  sub_network  x_pu  \
Line                         ...                                            
My line 0             False  ...       -inf        inf                0.0   
My line 1             False  ...       -inf        inf                0.0   
My line 2             False  ...       -inf        inf                0.0   

attribute  r_pu  g_pu  b_pu x_pu_eff  r_pu_eff  s_nom_opt  
Line                                                       
My line 0   0.0   0.0   0.0      0.0       0.0        0.0  
My line 1   0.0   0.0   0.0      0.0       0.0  

### Generator Creation

In [10]:
if "My gen" in network.generators.index:
    network.remove("Generator", "My gen")

network.add("Generator", "My gen",
            bus="My bus 0",
            p_set=100,  
            control="PQ")  

print(network.generators)
print()
print(network.generators.p_set)

attribute       bus control type  p_nom  p_nom_mod  p_nom_extendable  \
Generator                                                              
My gen     My bus 0      PQ         0.0        0.0             False   

attribute  p_nom_min  p_nom_max  p_min_pu  p_max_pu  ...  min_up_time  \
Generator                                            ...                
My gen           0.0        inf       0.0       1.0  ...            0   

attribute  min_down_time  up_time_before down_time_before  ramp_limit_up  \
Generator                                                                  
My gen                 0               1                0            NaN   

attribute  ramp_limit_down  ramp_limit_start_up  ramp_limit_shut_down  weight  \
Generator                                                                       
My gen                 NaN                  1.0                   1.0     1.0   

attribute  p_nom_opt  
Generator             
My gen           0.0  

[1 rows x 34 columns

### Load Creation

In [12]:
for i in range(n_buses):
    network.add("Load", "My load {}".format(i),
                bus="My bus {}".format(i),
                p_set=50)  # Add a load of 50 MW to each bus

print(network.loads)

attribute       bus carrier type  p_set  q_set  sign
Load                                                
My load 0  My bus 0                50.0    0.0  -1.0
My load 1  My bus 1                50.0    0.0  -1.0
My load 2  My bus 2                50.0    0.0  -1.0


## Newton - Raphson Power Flow
In the Newton-Raphson power flow we use Newton's Method to determine the voltage magnitude and angle at. each bus in the power system that satisfies power balance

The Objective of a power flow study is to calculate the voltage (magnitude and angle) for a given load, generation, and network condition. Once voltages are known for all buses, line flows and losses can be calculated  

### Performing Power Flow Calculation and Printing Results

In [13]:
network.pf()

print("Active Power for lines (p0):")
print(network.lines_t.p0)
print()
print("Voltage Angles for buses (degrees):")
print(network.buses_t.v_ang * 180 / np.pi)
print()
print("Voltage Magnitudes for buses (pu):")
print(network.buses_t.v_mag_pu)

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.124919 seconds


Active Power for lines (p0):
          My line 0     My line 1  My line 2
snapshot                                    
now       50.062667 -5.676337e-14      -50.0

Voltage Angles for buses (degrees):
Bus       My bus 0  My bus 1  My bus 2
snapshot                              
now            0.0  -0.71717  -0.71717

Voltage Magnitudes for buses (pu):
Bus       My bus 0  My bus 1  My bus 2
snapshot                              
now            1.0   0.99867   0.99867
